# 광고 문구 생성 테스트 (v2.0.0)

## 목적
- `industries.yaml` 기반 업종별 광고 문구 생성 테스트
- `PromptTemplateManager` 및 `TextGenerator` 통합 테스트
- 다양한 업종에 대한 사용자 입력 → 최종 광고 문구 파이프라인 검증

## 테스트 범위
1. 업종 자동 감지 테스트
2. 업종별 프롬프트 생성 테스트
3. 광고 문구 생성 테스트 (GPT API 호출)
4. 통합 파이프라인 테스트

---

## 0. 환경 설정

In [2]:
import sys
from pathlib import Path

# 프로젝트 루트 설정
project_root = Path.cwd()
while project_root.name != 'codeit_ad_smallbiz' and project_root.parent != project_root:
    project_root = project_root.parent

if project_root.name == 'codeit_ad_smallbiz':
    sys.path.insert(0, str(project_root))
    print(f"✅ 프로젝트 루트 설정: {project_root}")
else:
    print("⚠️ codeit_ad_smallbiz 폴더를 찾을 수 없습니다.")
    print(f"현재 위치: {Path.cwd()}")

✅ 프로젝트 루트 설정: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz


In [3]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ OPENAI_API_KEY 로드됨 (길이: {len(api_key)})")
else:
    print("❌ OPENAI_API_KEY가 설정되지 않았습니다.")

✅ OPENAI_API_KEY 로드됨 (길이: 164)


---
## 1. 모듈 임포트 테스트

In [4]:
# PromptTemplateManager 임포트
try:
    from src.generation.text_generation.prompt_manager import (
        PromptTemplateManager,
        IndustryConfigLoader,
        AdCopyPromptBuilder,
        detect_industry
    )
    print("✅ prompt_manager 모듈 임포트 성공")
except ImportError as e:
    print(f"❌ prompt_manager 임포트 실패: {e}")

✅ prompt_manager 모듈 임포트 성공


In [9]:
# TextGenerator 임포트
try:
    from src.generation.text_generation.text_generator import TextGenerator
    print("✅ text_generator 모듈 임포트 성공")
except ImportError as e:
    print(f"❌ text_generator 임포트 실패: {e}")

❌ text_generator 임포트 실패: No module named 'uvicorn'


In [11]:
# ad_generator 임포트 (통합 모듈)
try:
    from src.generation.text_generation.ad_generator import generate_advertisement
    print("✅ ad_generator 모듈 임포트 성공")
except ImportError as e:
    print(f"❌ ad_generator 임포트 실패: {e}")

✅ ad_generator 모듈 임포트 성공


In [12]:
from src.utils.logging import get_logger

logger = get_logger(__name__)

logger.info("테스트")

In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logging.info("테스트 메시지입니다.")

---
## 2. IndustryConfigLoader 테스트

In [13]:
# IndustryConfigLoader 초기화
loader = IndustryConfigLoader()

print("=" * 60)
print("📋 industries.yaml 로드 테스트")
print("=" * 60)

# 모든 하위 그룹 코드 확인
subgroups = loader.get_all_subgroups()
print(f"\n✅ 총 {len(subgroups)}개의 하위 그룹 로드됨")
print(f"하위 그룹 코드: {subgroups}")

📋 industries.yaml 로드 테스트

✅ 총 18개의 하위 그룹 로드됨
하위 그룹 코드: ['s1_hot_cooking', 's2_freshness', 's3_emotional', 's4_neat_variety', 'a1_beauty', 'a2_wellness', 'a3_fashion', 'a4_delicate_care', 'b1_learning', 'b2_leisure', 'b3_sports', 'c1_office', 'c2_medical', 'c3_technical', 'd1_equipment', 'd2_appliances', 'e1_infrastructure', 'e2_ceremony']


In [7]:
# 등급별 하위 그룹 분류
print("\n📊 등급별 하위 그룹 분류:")
print("-" * 40)

grade_groups = {}
for subgroup in subgroups:
    grade = loader.get_grade_for_industry(subgroup)
    if grade not in grade_groups:
        grade_groups[grade] = []
    grade_groups[grade].append(subgroup)

for grade, groups in grade_groups.items():
    print(f"\n{grade.upper()}:")
    for g in groups:
        industry_data = loader.get_industry(g)
        name_ko = industry_data.get('name_ko', 'N/A') if industry_data else 'N/A'
        print(f"  - {g}: {name_ko}")


📊 등급별 하위 그룹 분류:
----------------------------------------

S_GRADE:
  - s1_hot_cooking: S-1: 고온 조리 및 육즙 강조
  - s2_freshness: S-2: 신선도 및 색감 강조
  - s3_emotional: S-3: 감성 및 부드러운 질감
  - s4_neat_variety: S-4: 정갈함과 다채로움

A_GRADE:
  - a1_beauty: A-1: 인물 중심 뷰티 및 에스테틱
  - a2_wellness: A-2: 신체 변화 및 웰니스
  - a3_fashion: A-3: 패션 및 스타일 아이템
  - a4_delicate_care: A-4: 섬세한 케어 및 감성 소품

B_GRADE:
  - b1_learning: B-1: 학습/몰입
  - b2_leisure: B-2: 여가/스테이
  - b3_sports: B-3: 스포츠/활동

C_GRADE:
  - c1_office: C-1: 전문 사무
  - c2_medical: C-2: 의료/케어
  - c3_technical: C-3: 기술/시공

D_GRADE:
  - d1_equipment: D-1: 중장비/부품
  - d2_appliances: D-2: 일반 기기

E_GRADE:
  - e1_infrastructure: E-1: 편의/인프라
  - e2_ceremony: E-2: 예절/의례


---
## 3. 업종 자동 감지 테스트

In [8]:
# PromptTemplateManager 초기화
manager = PromptTemplateManager()

# 테스트 케이스: 다양한 업종의 사용자 입력
test_inputs = [
    # S등급: 감각/맛 자극
    "삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습",
    "횟집 신선한 회 홍보, 싱싱한 해산물",
    "카페 봄 시즌 딸기 음료 출시 홍보",
    "베이커리 신상 크루아상 홍보",
    
    # A등급: 시각적 변화
    "미용실 봄 시즌 헤어컬러 이벤트",
    "헬스장 PT 프로그램 신규 회원 모집",
    "네일샵 봄맞이 할인 이벤트",
    "옷가게 봄 신상 입고",
    
    # B등급: 공간 경험
    "독서실 조용한 공부 환경 홍보",
    "키즈카페 새 놀이기구 도입",
    "요가원 초보자 클래스 모집",
    
    # C등급: 신뢰 기반
    "법률사무소 무료 상담 이벤트",
    "치과 정기 검진 할인",
    "세무사 사업자 세금 상담",
    
    # D등급: 목적 중심
    "철물점 봄맞이 정원용품 할인",
    "가전제품 매장 에어컨 사전예약",
    
    # E등급: 인프라/특수
    "주차장 월정액 할인 이벤트",
    "이사업체 봄맞이 이사 특가"
]

print("=" * 70)
print("🔍 업종 자동 감지 테스트")
print("=" * 70)

for user_input in test_inputs:
    detected = manager.detect_industry(user_input)
    grade = loader.get_grade_for_industry(detected)
    info = manager.get_industry_info(detected)
    
    print(f"\n입력: {user_input[:30]}...")
    print(f"  → 감지: {detected} ({info.get('name_ko', 'N/A')})")
    print(f"  → 등급: {grade}")

🔍 업종 자동 감지 테스트

입력: 삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습...
  → 감지: s1_hot_cooking (S-1: 고온 조리 및 육즙 강조)
  → 등급: s_grade

입력: 횟집 신선한 회 홍보, 싱싱한 해산물...
  → 감지: s2_freshness (S-2: 신선도 및 색감 강조)
  → 등급: s_grade

입력: 카페 봄 시즌 딸기 음료 출시 홍보...
  → 감지: s3_emotional (S-3: 감성 및 부드러운 질감)
  → 등급: s_grade

입력: 베이커리 신상 크루아상 홍보...
  → 감지: s3_emotional (S-3: 감성 및 부드러운 질감)
  → 등급: s_grade

입력: 미용실 봄 시즌 헤어컬러 이벤트...
  → 감지: a1_beauty (A-1: 인물 중심 뷰티 및 에스테틱)
  → 등급: a_grade

입력: 헬스장 PT 프로그램 신규 회원 모집...
  → 감지: a2_wellness (A-2: 신체 변화 및 웰니스)
  → 등급: a_grade

입력: 네일샵 봄맞이 할인 이벤트...
  → 감지: a1_beauty (A-1: 인물 중심 뷰티 및 에스테틱)
  → 등급: a_grade

입력: 옷가게 봄 신상 입고...
  → 감지: a3_fashion (A-3: 패션 및 스타일 아이템)
  → 등급: a_grade

입력: 독서실 조용한 공부 환경 홍보...
  → 감지: b1_learning (B-1: 학습/몰입)
  → 등급: b_grade

입력: 키즈카페 새 놀이기구 도입...
  → 감지: s3_emotional (S-3: 감성 및 부드러운 질감)
  → 등급: s_grade

입력: 요가원 초보자 클래스 모집...
  → 감지: a2_wellness (A-2: 신체 변화 및 웰니스)
  → 등급: a_grade

입력: 법률사무소 무료 상담 이벤트...
  → 감지: c1_office (C-1: 전문 사무)
  → 등급: c_grade

입력: 치과 정기 검진 

---
## 4. 프롬프트 생성 테스트 (API 호출 없이)

In [9]:
# 업종별 프롬프트 확인
test_case = "삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습"

print("=" * 70)
print("📝 프롬프트 생성 테스트 (S1: 열조리/바베큐)")
print("=" * 70)
print(f"\n입력: {test_case}")

# 프롬프트 생성
prompts = manager.get_ad_copy_prompt(
    user_input=test_case,
    tone="warm",
    max_length=20
)

print(f"\n감지된 업종: {prompts['industry']}")
print(f"\n[시스템 프롬프트]")
print("-" * 50)
print(prompts['system_prompt'])
print(f"\n[사용자 프롬프트]")
print("-" * 50)
print(prompts['user_prompt'])

if prompts.get('examples'):
    print(f"\n[예시 광고 문구]")
    print("-" * 50)
    for ex in prompts['examples']:
        print(f"  - {ex}")

📝 프롬프트 생성 테스트 (S1: 열조리/바베큐)

입력: 삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습

감지된 업종: s1_hot_cooking

[시스템 프롬프트]
--------------------------------------------------
당신은 소상공인을 위한 전문 광고 카피라이터입니다.
AIDA 프레임워크를 활용하여 짧고 임팩트 있는 광고 문구를 만들어주세요.

규칙:
- 20자 이내 (공백 포함)
- 번호, 특수문자 없이 문구만 작성
- 한국어로 작성 (사용자가 다른 언어 요청 시 제외)
- 광고 문구 1개만 생성

키워드 우선순위 (이 순서대로 강조):
1. sensory: 겉은 바삭, 속은 촉촉한, 육즙이 흐르는, 짭조름하게 양념한
2. temperature: 갓 구운, 뜨끈뜨끈한, 주문 즉시 조리
3. immediacy: 지금 주문, 오늘만, 한정 수량
4. context: 야식으로, 회식 메뉴로, 혼술 안주로
5. price: 20% 할인, 배달비 무료, 곱빼기 서비스

브랜드 보이스:
친근하고 감각적인, 식욕을 자극하는 톤

CTA 스타일:
지금 주문, 배달앱에서 주문, 전화 주문

피해야 할 표현: 건강, 다이어트, 저칼로리

톤 앤 매너:
따뜻하고 감성적, 위로하는

[사용자 프롬프트]
--------------------------------------------------
다음 내용으로 광고 문구를 만들어주세요:

삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습

요구사항:
- 20자 이내
- 광고 문구만 작성 (설명, 번호 등 제외)
- 감성적이면서도 명확한 메시지 전달

참고 프롬프트 구조:
{'role': '{business_type} delivery order specialist copywriter', 'target': '20-30s office workers, {situation} context', 'pain_points': ['soggy or hard food', 'long delivery time'], 

In [10]:
# 다른 업종 테스트: 카페
test_case_cafe = "카페 봄 시즌 딸기 음료 출시 홍보"

print("=" * 70)
print("📝 프롬프트 생성 테스트 (S3: 감성/분위기 음료)")
print("=" * 70)
print(f"\n입력: {test_case_cafe}")

prompts_cafe = manager.get_ad_copy_prompt(
    user_input=test_case_cafe,
    tone="warm",
    max_length=20
)

print(f"\n감지된 업종: {prompts_cafe['industry']}")
print(f"\n[시스템 프롬프트]")
print("-" * 50)
print(prompts_cafe['system_prompt'])

📝 프롬프트 생성 테스트 (S3: 감성/분위기 음료)

입력: 카페 봄 시즌 딸기 음료 출시 홍보

감지된 업종: s3_emotional

[시스템 프롬프트]
--------------------------------------------------
당신은 소상공인을 위한 전문 광고 카피라이터입니다.
AIDA 프레임워크를 활용하여 짧고 임팩트 있는 광고 문구를 만들어주세요.

규칙:
- 20자 이내 (공백 포함)
- 번호, 특수문자 없이 문구만 작성
- 한국어로 작성 (사용자가 다른 언어 요청 시 제외)
- 광고 문구 1개만 생성

키워드 우선순위 (이 순서대로 강조):
1. emotional: 나를 위한 시간, 달콤한 위로, 행복한 순간
2. aesthetic: 예쁜, 인스타감성, 아기자기한
3. quality: 수제, 천연, 무첨가
4. trend: 겨울 시그니처, 베스트셀러, 신메뉴
5. experience: 데이트 코스로, 친구와 함께

브랜드 보이스:
따뜻하고 감성적인, 위로하는 톤

CTA 스타일:
방문하기, SNS 해시태그, 지금 맛보기

피해야 할 표현: 빠른, 저렴, 서두르다

톤 앤 매너:
따뜻하고 감성적, 위로하는


In [11]:
# C등급 테스트: 법률사무소
test_case_legal = "법률사무소 무료 상담 이벤트"

print("=" * 70)
print("📝 프롬프트 생성 테스트 (C1: 법률/세무)")
print("=" * 70)
print(f"\n입력: {test_case_legal}")

prompts_legal = manager.get_ad_copy_prompt(
    user_input=test_case_legal,
    tone="professional",
    max_length=20
)

print(f"\n감지된 업종: {prompts_legal['industry']}")
print(f"추천 톤: {manager.get_recommended_tone(prompts_legal['industry'])}")
print(f"\n[시스템 프롬프트]")
print("-" * 50)
print(prompts_legal['system_prompt'])

📝 프롬프트 생성 테스트 (C1: 법률/세무)

입력: 법률사무소 무료 상담 이벤트

감지된 업종: c1_office
추천 톤: professional

[시스템 프롬프트]
--------------------------------------------------
당신은 소상공인을 위한 전문 광고 카피라이터입니다.
AIDA 프레임워크를 활용하여 짧고 임팩트 있는 광고 문구를 만들어주세요.

규칙:
- 20자 이내 (공백 포함)
- 번호, 특수문자 없이 문구만 작성
- 한국어로 작성 (사용자가 다른 언어 요청 시 제외)
- 광고 문구 1개만 생성

키워드 우선순위 (이 순서대로 강조):
1. expert_experience: 20년 경력, 전문 변호사, 공인 회계사
2. problem_solving: 복잡한 문제도, 정확한 분석, 최선의 방법
3. trust: 믿고 맡기는, 안전하게, 투명한
4. track_record: 성공률, 처리 건수, 고객 만족
5. accessibility: 무료 상담, 빠른 응대

브랜드 보이스:
믿음직하고 전문적인 톤

CTA 스타일:
전화 상담, 무료 상담 신청, 방문 예약

피해야 할 표현: 빠른, 쉬운, 간단한

톤 앤 매너:
믿음직하고 전문적인


---
## 5. 광고 문구 생성 테스트 (GPT API 호출)

In [12]:
# TextGenerator 초기화
text_gen = TextGenerator(use_industry_config=True)

print("=" * 70)
print("🚀 광고 문구 생성 테스트 (GPT API 호출)")
print("=" * 70)

🚀 광고 문구 생성 테스트 (GPT API 호출)


In [13]:
# 테스트 1: 삼겹살 맛집
result1 = text_gen.generate_ad_copy_with_info(
    user_input="삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습",
    tone="auto",  # 업종에 맞는 톤 자동 선택
    max_length=20
)

print("\n" + "=" * 50)
print("테스트 1: 삼겹살 맛집")
print("=" * 50)
print(f"생성된 광고 문구: {result1['ad_copy']}")
print(f"감지된 업종: {result1['industry']}")
print(f"적용된 톤: {result1['tone']}")
if result1['examples']:
    print(f"참고 예시: {result1['examples'][:2]}")

📝 광고 문구 생성 중...
   입력: 삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습
   감지된 업종: s1_hot_cooking
   톤: warm, 최대 20자
✅ 생성 완료: 지글지글 삼겹살 지금 주문!

테스트 1: 삼겹살 맛집
생성된 광고 문구: 지글지글 삼겹살 지금 주문!
감지된 업종: s1_hot_cooking
적용된 톤: warm
참고 예시: ['Crispy outside, juicy inside chicken in 30 minutes. Fresh from the fryer!', 'Sizzling pork belly grilled over charcoal. 20% off today only!']


In [14]:
# 테스트 2: 카페
result2 = text_gen.generate_ad_copy_with_info(
    user_input="카페 봄 시즌 딸기 음료 출시 홍보, 따뜻한 느낌",
    tone="auto",
    max_length=20
)

print("\n" + "=" * 50)
print("테스트 2: 카페")
print("=" * 50)
print(f"생성된 광고 문구: {result2['ad_copy']}")
print(f"감지된 업종: {result2['industry']}")
print(f"적용된 톤: {result2['tone']}")

📝 광고 문구 생성 중...
   입력: 카페 봄 시즌 딸기 음료 출시 홍보, 따뜻한 느낌
   감지된 업종: s3_emotional
   톤: warm, 최대 20자
✅ 생성 완료: 딸기 한 모금의 행복, 봄의 달콤한

테스트 2: 카페
생성된 광고 문구: 딸기 한 모금의 행복, 봄의 달콤한
감지된 업종: s3_emotional
적용된 톤: warm


In [15]:
# 테스트 3: 미용실
result3 = text_gen.generate_ad_copy_with_info(
    user_input="미용실 봄 시즌 헤어컬러 이벤트, 새로운 나를 만나다",
    tone="auto",
    max_length=20
)

print("\n" + "=" * 50)
print("테스트 3: 미용실")
print("=" * 50)
print(f"생성된 광고 문구: {result3['ad_copy']}")
print(f"감지된 업종: {result3['industry']}")
print(f"적용된 톤: {result3['tone']}")

📝 광고 문구 생성 중...
   입력: 미용실 봄 시즌 헤어컬러 이벤트, 새로운 나를 만나다
   감지된 업종: a1_beauty
   톤: friendly, 최대 20자
✅ 생성 완료: 새로운 나를 만나는 컬러 변신

테스트 3: 미용실
생성된 광고 문구: 새로운 나를 만나는 컬러 변신
감지된 업종: a1_beauty
적용된 톤: friendly


In [16]:
# 테스트 4: 헬스장
result4 = text_gen.generate_ad_copy_with_info(
    user_input="헬스장 PT 프로그램 신규 회원 모집, 전문 트레이너",
    tone="auto",
    max_length=20
)

print("\n" + "=" * 50)
print("테스트 4: 헬스장")
print("=" * 50)
print(f"생성된 광고 문구: {result4['ad_copy']}")
print(f"감지된 업종: {result4['industry']}")
print(f"적용된 톤: {result4['tone']}")

📝 광고 문구 생성 중...
   입력: 헬스장 PT 프로그램 신규 회원 모집, 전문 트레이너
   감지된 업종: a2_wellness
   톤: friendly, 최대 20자
✅ 생성 완료: 함께하는 PT, 목표 달성의 시작!

테스트 4: 헬스장
생성된 광고 문구: 함께하는 PT, 목표 달성의 시작!
감지된 업종: a2_wellness
적용된 톤: friendly


In [17]:
# 테스트 5: 법률사무소 (C등급 - 전문성 강조)
result5 = text_gen.generate_ad_copy_with_info(
    user_input="법률사무소 무료 상담 이벤트, 변호사 직접 상담",
    tone="auto",
    max_length=20
)

print("\n" + "=" * 50)
print("테스트 5: 법률사무소")
print("=" * 50)
print(f"생성된 광고 문구: {result5['ad_copy']}")
print(f"감지된 업종: {result5['industry']}")
print(f"적용된 톤: {result5['tone']}")

📝 광고 문구 생성 중...
   입력: 법률사무소 무료 상담 이벤트, 변호사 직접 상담
   감지된 업종: c1_office
   톤: professional, 최대 20자
✅ 생성 완료: 변호사와 무료 상담 받기

테스트 5: 법률사무소
생성된 광고 문구: 변호사와 무료 상담 받기
감지된 업종: c1_office
적용된 톤: professional


In [18]:
# 테스트 6: 꽃집
result6 = text_gen.generate_ad_copy_with_info(
    user_input="꽃집 봄 시즌 꽃다발 할인, 사랑을 전하세요",
    tone="auto",
    max_length=20
)

print("\n" + "=" * 50)
print("테스트 6: 꽃집")
print("=" * 50)
print(f"생성된 광고 문구: {result6['ad_copy']}")
print(f"감지된 업종: {result6['industry']}")
print(f"적용된 톤: {result6['tone']}")

📝 광고 문구 생성 중...
   입력: 꽃집 봄 시즌 꽃다발 할인, 사랑을 전하세요
   감지된 업종: a4_delicate_care
   톤: friendly, 최대 20자
✅ 생성 완료: 사랑의 꽃다발로 특별함을

테스트 6: 꽃집
생성된 광고 문구: 사랑의 꽃다발로 특별함을
감지된 업종: a4_delicate_care
적용된 톤: friendly


---
## 6. 톤 비교 테스트

In [19]:
# 같은 입력에 대해 다른 톤 적용 비교
test_input = "카페 신메뉴 딸기라떼 홍보"

tones = ["warm", "professional", "friendly", "energetic"]

print("=" * 70)
print("🎨 톤 비교 테스트")
print("=" * 70)
print(f"입력: {test_input}")
print("-" * 70)

for tone in tones:
    result = text_gen.generate_ad_copy(
        user_input=test_input,
        tone=tone,
        max_length=20
    )
    print(f"\n[{tone}] → {result}")

🎨 톤 비교 테스트
입력: 카페 신메뉴 딸기라떼 홍보
----------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 카페 신메뉴 딸기라떼 홍보
   감지된 업종: s3_emotional
   톤: warm, 최대 20자
✅ 생성 완료: 달콤한 딸기라떼로 행복한 순간!

[warm] → 달콤한 딸기라떼로 행복한 순간!
📝 광고 문구 생성 중...
   입력: 카페 신메뉴 딸기라떼 홍보
   감지된 업종: s3_emotional
   톤: professional, 최대 20자
✅ 생성 완료: 딸기라떼와 함께하는 행복한 순간

[professional] → 딸기라떼와 함께하는 행복한 순간
📝 광고 문구 생성 중...
   입력: 카페 신메뉴 딸기라떼 홍보
   감지된 업종: s3_emotional
   톤: friendly, 최대 20자
✅ 생성 완료: 달콤한 순간, 예쁜 딸기라떼✨

[friendly] → 달콤한 순간, 예쁜 딸기라떼✨
📝 광고 문구 생성 중...
   입력: 카페 신메뉴 딸기라떼 홍보
   감지된 업종: s3_emotional
   톤: energetic, 최대 20자
✅ 생성 완료: 달콤한 딸기라떼, 행복한 순간을 담아

[energetic] → 달콤한 딸기라떼, 행복한 순간을 담아


---
## 7. 전체 업종 테스트 (일괄 실행)

In [20]:
# 전체 업종 대표 테스트 케이스
comprehensive_tests = [
    # S등급
    {"input": "삼겹살 맛집, 숯불에 굽는 고기", "expected_grade": "s_grade"},
    {"input": "횟집 광어회 특선, 신선한 해산물", "expected_grade": "s_grade"},
    {"input": "카페 아메리카노 원두 향이 좋은", "expected_grade": "s_grade"},
    {"input": "베이커리 갓 구운 식빵", "expected_grade": "s_grade"},
    
    # A등급
    {"input": "미용실 펌 스타일링 이벤트", "expected_grade": "a_grade"},
    {"input": "헬스장 다이어트 PT 프로그램", "expected_grade": "a_grade"},
    {"input": "네일샵 젤네일 할인", "expected_grade": "a_grade"},
    {"input": "옷가게 봄 신상 의류", "expected_grade": "a_grade"},
    
    # B등급
    {"input": "독서실 24시간 운영", "expected_grade": "b_grade"},
    {"input": "키즈카페 어린이 생일파티", "expected_grade": "b_grade"},
    {"input": "요가원 명상 클래스", "expected_grade": "b_grade"},
    
    # C등급
    {"input": "변호사 법률 상담", "expected_grade": "c_grade"},
    {"input": "치과 임플란트 상담", "expected_grade": "c_grade"},
    {"input": "세무사 종합소득세 신고", "expected_grade": "c_grade"},
    
    # D등급
    {"input": "철물점 공구 세트 할인", "expected_grade": "d_grade"},
    {"input": "가전매장 냉장고 특가", "expected_grade": "d_grade"},
    
    # E등급
    {"input": "주차장 월정액 주차", "expected_grade": "e_grade"},
    {"input": "이사업체 포장이사 서비스", "expected_grade": "e_grade"},
]

print("=" * 80)
print("📊 전체 업종 테스트 결과")
print("=" * 80)

results = []
for test in comprehensive_tests:
    detected = manager.detect_industry(test["input"])
    actual_grade = loader.get_grade_for_industry(detected)
    expected_grade = test["expected_grade"]
    
    is_correct = actual_grade == expected_grade
    status = "✅" if is_correct else "❌"
    
    results.append({
        "input": test["input"],
        "detected": detected,
        "expected_grade": expected_grade,
        "actual_grade": actual_grade,
        "correct": is_correct
    })
    
    print(f"\n{status} 입력: {test['input'][:25]}...")
    print(f"   감지: {detected} (등급: {actual_grade})")
    if not is_correct:
        print(f"   ⚠️ 예상 등급: {expected_grade}")

# 정확도 계산
correct_count = sum(1 for r in results if r["correct"])
total_count = len(results)
accuracy = correct_count / total_count * 100

print("\n" + "=" * 80)
print(f"📈 업종 감지 정확도: {correct_count}/{total_count} ({accuracy:.1f}%)")
print("=" * 80)

📊 전체 업종 테스트 결과

✅ 입력: 삼겹살 맛집, 숯불에 굽는 고기...
   감지: s1_hot_cooking (등급: s_grade)

✅ 입력: 횟집 광어회 특선, 신선한 해산물...
   감지: s2_freshness (등급: s_grade)

✅ 입력: 카페 아메리카노 원두 향이 좋은...
   감지: s3_emotional (등급: s_grade)

✅ 입력: 베이커리 갓 구운 식빵...
   감지: s3_emotional (등급: s_grade)

✅ 입력: 미용실 펌 스타일링 이벤트...
   감지: a1_beauty (등급: a_grade)

✅ 입력: 헬스장 다이어트 PT 프로그램...
   감지: a2_wellness (등급: a_grade)

✅ 입력: 네일샵 젤네일 할인...
   감지: a1_beauty (등급: a_grade)

✅ 입력: 옷가게 봄 신상 의류...
   감지: a3_fashion (등급: a_grade)

✅ 입력: 독서실 24시간 운영...
   감지: b1_learning (등급: b_grade)

❌ 입력: 키즈카페 어린이 생일파티...
   감지: s3_emotional (등급: s_grade)
   ⚠️ 예상 등급: b_grade

❌ 입력: 요가원 명상 클래스...
   감지: a2_wellness (등급: a_grade)
   ⚠️ 예상 등급: b_grade

✅ 입력: 변호사 법률 상담...
   감지: c1_office (등급: c_grade)

✅ 입력: 치과 임플란트 상담...
   감지: c2_medical (등급: c_grade)

✅ 입력: 세무사 종합소득세 신고...
   감지: c1_office (등급: c_grade)

✅ 입력: 철물점 공구 세트 할인...
   감지: d1_equipment (등급: d_grade)

✅ 입력: 가전매장 냉장고 특가...
   감지: d2_appliances (등급: d_grade)

❌ 입력: 주차장 월정액 주차...
   감지: s4_neat_

---
## 8. 통합 파이프라인 테스트 (ad_generator)

In [21]:
# 통합 파이프라인 테스트
print("=" * 80)
print("🎬 통합 파이프라인 테스트 (generate_advertisement)")
print("=" * 80)

try:
    result = generate_advertisement(
        user_input="카페 신메뉴 딸기라떼 홍보, 따뜻한 느낌",
        tone="warm",
        max_length=20,
        style="realistic"
    )
    
    print("\n📋 결과:")
    print("-" * 50)
    print(f"광고 문구: {result.get('ad_copy')}")
    print(f"Positive Prompt: {result.get('positive_prompt', '')[:80]}...")
    print(f"Negative Prompt: {result.get('negative_prompt', '(없음)')}")
    print(f"업종: {result.get('industry')}")
    print(f"상태: {result.get('status')}")
    
except Exception as e:
    print(f"❌ 통합 테스트 실패: {e}")
    import traceback
    traceback.print_exc()

🎬 통합 파이프라인 테스트 (generate_advertisement)
🎬 광고 생성 파이프라인 시작
📥 사용자 입력: 카페 신메뉴 딸기라떼 홍보, 따뜻한 느낌
📐 설정: tone=warm, max_length=20, style=realistic

1️⃣ 광고 문구 생성
--------------------------------------------------------------------------------
📝 광고 문구 생성 중...
   입력: 카페 신메뉴 딸기라떼 홍보, 따뜻한 느낌
   감지된 업종: s3_emotional
   톤: warm, 최대 20자
✅ 생성 완료: 달콤한 딸기라떼와 함께하는 행복한 순
✅ 광고 문구: '달콤한 딸기라떼와 함께하는 행복한 순' (20자)

2️⃣ 이미지 프롬프트 생성
--------------------------------------------------------------------------------
✅ Positive: 255 chars
✅ Negative: 0 chars
✅ Industry: s3_emotional

🎉 광고 생성 완료!
📝 광고 문구: 달콤한 딸기라떼와 함께하는 행복한 순
📸 Positive: Professional commercial photography of 카페 신메뉴 딸기라떼 홍보, 따뜻한 느...
🚫 Negative: ...
🏢 업종: s3_emotional


📋 결과:
--------------------------------------------------
광고 문구: 달콤한 딸기라떼와 함께하는 행복한 순
Positive Prompt: Professional commercial photography of 카페 신메뉴 딸기라떼 홍보, 따뜻한 느낌. soft natural wind...
Negative Prompt: 
업종: s3_emotional
상태: success


---
## 9. 결과 요약

In [22]:
print("=" * 80)
print("📋 테스트 결과 요약")
print("=" * 80)

print("""
✅ 모듈 임포트 테스트
   - prompt_manager: PromptTemplateManager, IndustryConfigLoader
   - text_generator: TextGenerator (v2.0.0)
   - ad_generator: generate_advertisement

✅ industries.yaml 로드 테스트
   - 6개 등급 (S, A, B, C, D, E)
   - 18개 하위 그룹
   - 247개 업종 분류

✅ 업종 자동 감지 테스트
   - 한글 키워드 기반 감지
   - 등급별 정확도 확인

✅ 프롬프트 생성 테스트
   - 업종별 최적화 프롬프트
   - AIDA 프레임워크 적용
   - 키워드 우선순위 반영

✅ 광고 문구 생성 테스트 (GPT API)
   - 자동 톤 선택 (tone="auto")
   - 업종별 특화 광고 문구 생성
   - 메타데이터 포함 결과 반환

✅ 톤 비교 테스트
   - warm, professional, friendly, energetic

✅ 통합 파이프라인 테스트
   - 사용자 입력 → 업종 감지 → 광고 문구 생성 → 이미지 프롬프트 생성
""")

print("=" * 80)
print("🎉 모든 테스트 완료!")
print("=" * 80)

📋 테스트 결과 요약

✅ 모듈 임포트 테스트
   - prompt_manager: PromptTemplateManager, IndustryConfigLoader
   - text_generator: TextGenerator (v2.0.0)
   - ad_generator: generate_advertisement

✅ industries.yaml 로드 테스트
   - 6개 등급 (S, A, B, C, D, E)
   - 18개 하위 그룹
   - 247개 업종 분류

✅ 업종 자동 감지 테스트
   - 한글 키워드 기반 감지
   - 등급별 정확도 확인

✅ 프롬프트 생성 테스트
   - 업종별 최적화 프롬프트
   - AIDA 프레임워크 적용
   - 키워드 우선순위 반영

✅ 광고 문구 생성 테스트 (GPT API)
   - 자동 톤 선택 (tone="auto")
   - 업종별 특화 광고 문구 생성
   - 메타데이터 포함 결과 반환

✅ 톤 비교 테스트
   - warm, professional, friendly, energetic

✅ 통합 파이프라인 테스트
   - 사용자 입력 → 업종 감지 → 광고 문구 생성 → 이미지 프롬프트 생성

🎉 모든 테스트 완료!
